In [1]:
# lets take a deeper look.
import arrayfire as af

layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()

MHAttention_v = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_v").to_ndarray()
MHAttention_result_attn = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_result_attn").to_ndarray()
MHAttention_result_result = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_result_result").to_ndarray()
MHAttention_result_result_2 = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_result_result_2").to_ndarray()

selfAttention_result = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_result").to_ndarray()
selfAttention_wf_ = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_wf_").to_ndarray()
selfAttention_result_2 = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_result_2").to_ndarray()

layer_selfAttnResult = af.array.read_array("OUTPUT_TRF.arr", key="layer_selfAttnResult").to_ndarray()
layer_norm1_w = af.array.read_array("OUTPUT_TRF.arr", key="layer_norm1_w").to_ndarray()
layer_norm1_b = af.array.read_array("OUTPUT_TRF.arr", key="layer_norm1_b").to_ndarray()
layer_h = af.array.read_array("OUTPUT_TRF.arr", key="layer_h").to_ndarray()
layer_h_2 = af.array.read_array("OUTPUT_TRF.arr", key="layer_h_2").to_ndarray()

In [2]:
# testing again after chaning MCTCSelfOutput to this differnt way flashlight seems to do things

# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_attn = model.mctc.encoder.layers[0].attention


In [3]:
def valueprint(k, name=None):
    print(name, k.shape, k.sum(), k.std())
    
our_output = torch.Tensor(layer_x).unsqueeze(0).transpose(1,2)
valueprint(our_output, "INPUT")
our_output = our_attn(our_output)[0]
valueprint(our_output, "OUTPUT")


INPUT torch.Size([1, 321, 1536]) tensor(330.5734) tensor(0.1587)
selfattn_attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
selfattn_attention_probs_HEADS torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>) tensor(0.1639, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1023.2032, grad_fn=<SumBackward0>) tensor(0.0612, grad_fn=<StdBackward0>)
SelfOUtput input torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(22.9749, grad_fn=<SumBackward0>)
SelfOUtput input_tensor torch.Size([1, 321, 1536]) tensor(330.5734)
SelfOUtput postnorm torch.Size([1, 321, 1536]) tensor(-618.0341, grad_fn=<SumBackward0>)
OUTPUT torch.Size([1, 321, 1536]) tensor(-618.0341, grad_fn=<SumBackward0>) tensor(0.2111, grad_fn=<StdBackward0>)


In [4]:
'''1. QKV attention, then relative key position embedding then softmax for final 
attn values

selfattn_attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
'''
valueprint(MHAttention_result_attn)

None (321, 321, 4) 1283.9999 0.0022644862


In [5]:
'''2. Multiplying the above attn scores with the values vector

WARNING: even the stddev is the same, but the sum is off by a small value...

context_layer = torch.matmul(attention_probs, value_layer)

value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>) tensor(0.1639, grad_fn=<StdBackward0>)
attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1023.2032, grad_fn=<SumBackward0>) tensor(0.0612, grad_fn=<StdBackward0>)
'''
# the _2 part is just dimension change; exported it just to check in case
valueprint(MHAttention_v)
valueprint(MHAttention_result_attn)
valueprint(MHAttention_result_result_2)

None (321, 384, 4) -995.127 0.16386819
None (321, 321, 4) 1283.9999 0.0022644862
None (321, 1536) -1017.8952 0.061281413


```cpp
auto attn = dropout(softmax(scores, 1), pDropout);
arr = attn.array();
af::saveArray("MHAttention_result_attn", arr, savePathChar, true);

auto result = matmul(attn.as(v.type()), v);

arr = result.array();
af::saveArray("MHAttention_result_result", arr, savePathChar, true);
```

In [6]:
out_v = torch.Tensor(MHAttention_v).transpose(1,2).transpose(0,1)
out_attn = torch.Tensor(MHAttention_result_attn).transpose(1,2).transpose(0,1)
out_result_result = torch.matmul(out_attn, out_v)

valueprint(out_v, "out_v")
valueprint(out_attn, "out_attn")
valueprint(out_result_result)

out_v torch.Size([4, 321, 384]) tensor(-995.1271) tensor(0.1639)
out_attn torch.Size([4, 321, 321]) tensor(1283.9999) tensor(0.0023)
None torch.Size([4, 321, 384]) tensor(-1017.8952) tensor(0.0613)


In [10]:
# so it must be the case that the attn scores, despite being the sames sum, are wrong.
our_attn = model.mctc.encoder.layers[0].attention.self
our_output = torch.Tensor(layer_x).unsqueeze(0).transpose(1,2)
result, attn, value = our_attn(our_output, output_attentions=True)


selfattn_attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
selfattn_attention_probs_HEADS torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>) tensor(0.1639, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1023.2032, grad_fn=<SumBackward0>) tensor(0.0612, grad_fn=<StdBackward0>)


In [46]:
fl_attn_scores = out_attn[0]
fl_value = out_v[0]
fl_result = out_result_result[0]

our_attn_scores = attn.squeeze(0)[0] + 0.000013895
our_value = value.squeeze(0)[0]

our_manual = torch.matmul(our_attn_scores, our_value)
our_manual_test_attn = torch.matmul(our_attn_scores, fl_value)
our_manual_test_value = torch.matmul(fl_attn_scores, our_value)

valueprint(fl_result)
print()
valueprint(our_manual)
valueprint(our_manual_test_attn)
# valueprint(our_manual_test_value)



None torch.Size([321, 384]) tensor(-65.4505) tensor(0.0465)

None torch.Size([321, 384]) tensor(-65.4504, grad_fn=<SumBackward0>) tensor(0.0465, grad_fn=<StdBackward0>)
None torch.Size([321, 384]) tensor(-65.4505, grad_fn=<SumBackward0>) tensor(0.0465, grad_fn=<StdBackward0>)


In [9]:
'''3. wf_ multiply (SelfOUtput.dense) on above (attn*value) output

SelfOUtput input torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(22.9749, grad_fn=<SumBackward0>)
'''

# selfAttention_result_2 = (*wf_)(transpose(selfAttention_result));
valueprint(selfAttention_result)
valueprint(selfAttention_result_2)

None (321, 1536) -1017.8952 0.061281413
None (1536, 321) 25.706522 0.1094709
